In [3]:
###-------------------------------------------------
### Место, где мы изменяем важные перемнные

DIR_NAME = "Colab_demo"        # имя папки как она называется на диске
DATASET_FOLDER = "obj_1"       # название архива и одноименной папки в нём где хранятся наши фотографии и аннотации к ним

count_images = 2_000           #  количество фотографий которое мы хотим добавить (если в датасете меньше фотографий, то используется то количество которое имеется)

classes = ["drone"]            # список классов

width = 416                    # ширина и высота изображения (darknet автоматически преобразует фотографии в нужный размер)
height = 416



###------------------------------------------------

### Остальное ни в коем случае не трогать


%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/MyDrive/ /mydrive # это команда создать ссылку, так что длинный путь /content/gdrive/MyDrive/ будет равнозначен короткому и удобному /mydrive

%cd /mydrive


%cd "$DIR_NAME"


if os.path.exists("darknet") and os.path.isdir("darknet"):
  pass
else:
  !git clone https://github.com/AlexeyAB/darknet

%cd darknet

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29


!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

!/usr/local/cuda/bin/nvcc --version


!make

%cd ..



import os

folder_path = f"darknet/data/{DATASET_FOLDER}"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # ! rm -rf "$folder_path"
    pass
else:
  !unzip "$DATASET_FOLDER".zip -d darknet/data/






# !unzip "$DATASET_FOLDER".zip -d darknet/data/
%cd darknet/



from random import shuffle


validation_split = 0.2  # 0.2 - это значит, что 20% от всего датасета будет использоваться для валидации (можно менять, но рекомендуется оставить таким)


files = os.listdir(f"data/{DATASET_FOLDER}/")   #папка где хранятся фотографии и аннотации

count_images = min(count_images,( len(files) /2))

images = []

for file in files:
    if count_images >=0:
        if file.endswith('.jpg'):
            images.append(f"data/{DATASET_FOLDER}/" + file)
            count_images-=1

# images = [f"data/{DATASET_FOLDER}/" + file for file in files if file.endswith('.jpg')]


shuffle(images)

train_len = int((1-validation_split)*len(images))

train_dataset = images[:train_len]
valid_dataset = images[train_len:]

with open('data/train.txt', 'w') as f:
    txt = '\n'.join(images)
    f.write(txt)

with open('data/test.txt', 'w') as f:

  txt = '\n'.join(images)
  f.write(txt)




classes_amount = len(classes)

with open('data/obj.names', 'w') as f:
  f.write('\n'.join(classes))



if os.path.exists(f"/mydrive/{DIR_NAME}/backup") and os.path.isdir(f"/mydrive/{DIR_NAME}/backup"):
  pass
else:
  os.mkdir(f"/mydrive/{DIR_NAME}/backup")


train = 'train.txt'
valid = 'test.txt'      # дефолтные значения
names = 'obj.names'
backup = f'/mydrive/{DIR_NAME}/backup'   # папка куда будут сохраняться веса

txt = f"""classes={classes_amount}
train=data/{train}
valid=data/{valid}
names=data/{names}
backup={backup}"""

with open('data/obj.data', 'w') as f:
  f.write(txt)



yolov4_tiny = True

classes = classes_amount


filters = (classes +5 ) * 3

max_batches = classes * 3000
if max_batches < 6000:
  max_batches = 6000

step1 = int(max_batches * 0.8)
step2 = int(max_batches * 0.9)




import os


def copy_file(file, dest):
  os.system(f'cp {file} -d {dest}')


def change_in_file(file, old, new):
  os.system(f"sed -i 's/{old}/{new}/' {file}")


if yolov4_tiny:
  template_file = "cfg/yolov4-tiny-custom.cfg"
  config_file = "cfg/yolov4-tiny-obj.cfg"
  max_batches_line = "max_batches = 500200"




copy_file(template_file, config_file)  # Делаем копию файла конфигурации


change_in_file(config_file, "subdivisions=1", "subdivisions=16")
change_in_file(config_file, "width=416", f"width={width}")
change_in_file(config_file, "height=416", f"height={height}")
change_in_file(config_file, "random=0", "random=1")


change_in_file(config_file, "steps=400000,450000", f"steps={step1},{step2}")
change_in_file(config_file, "classes=80", f"classes={classes}")
change_in_file(config_file, "filters=255", f"filters={filters}")
change_in_file(config_file, max_batches_line, f"max_batches = {max_batches}")



!./darknet detector calc_anchors data/obj.data -num_of_clusters 6 -width "$width" -height "$height"



with open('anchors.txt', 'r', encoding='utf-8') as file:
    anchors = file.read()

new_anchors_str = f"anchors={anchors}"
os.system(f"sed -i 's/^anchors.*/{new_anchors_str}/' {config_file}")



!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -map

Выходные данные были обрезаны до нескольких последних строк (5000).
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.183125), count: 3, class_loss = 1.058100, iou_loss = 0.002131, total_loss = 1.060230 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.431913), count: 1, class_loss = 0.520787, iou_loss = 0.136933, total_loss = 0.657721 
 total_bbox = 14426, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.251577), count: 4, class_loss = 1.307817, iou_loss = 0.006784, total_loss = 1.314601 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.000000), count: 1, class_loss = 0.195296, iou_loss = 0.000000, total_loss = 0.195296 
 total_bbox = 14430, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.160975), count: 2, class_loss = 0.671551, iou_loss = 0.003020, total_loss = 0.